In [1]:
import os
import numpy as np
import torch
import time
import pickle
import matplotlib.pyplot as plt
from collections import OrderedDict
from torch.autograd import Variable
from pathlib import Path

%matplotlib inline

In [2]:
pix2pixhd_dir = Path('../src/pix2pixHD/')

import sys
sys.path.append(str(pix2pixhd_dir))

%load_ext autoreload
%autoreload 2

In [3]:
from options.train_options import TrainOptions
from data.data_loader import CreateDataLoader
from models.models import create_model
import util.util as util
from util.visualizer import Visualizer

In [4]:
with open('../data/train_opt.pkl', mode='rb') as f:
    opt = pickle.load(f)
    
iter_path = os.path.join(opt.checkpoints_dir, opt.name, 'iter.txt')

In [5]:
#opt.lr = 1e-6
opt.loadSize = 480
opt.label_nc = 0
opt.temp_data = False
opt.input_nc = 3
# opt.G_steps = 10
opt.no_flip = True
# opt.model = 'pix2pixHD_tempSmooth'
opt.debug = True
opt.name = 'maskGirl'
opt.dataroot='../data/target/maskGirl/'
# opt.load_pretrain = '../checkpoints/maskGirl/'

In [6]:
if opt.continue_train:
    try:
        start_epoch, epoch_iter = np.loadtxt(iter_path , delimiter=',', dtype=int)
    except:
        start_epoch, epoch_iter = 1, 0
    print('Resuming from epoch %d at iteration %d' % (start_epoch, epoch_iter))        
else:    
    start_epoch, epoch_iter = 1, 0

if opt.debug:
    opt.display_freq = 1
    opt.print_freq = 1
    opt.niter = 40
    opt.niter_decay = 40
    opt.max_dataset_size = 10
print(opt)

Namespace(batchSize=1, beta1=0.5, checkpoints_dir='../checkpoints/', continue_train=False, data_type=32, dataroot='../data/target/maskGirl/', debug=True, display_freq=1, display_winsize=512, feat_num=3, fineSize=512, fine_size=480, gpu_ids=[0], input_nc=3, instance_feat=False, isTrain=True, label_feat=False, label_nc=0, lambda_feat=10.0, loadSize=480, load_features=False, load_pretrain='', lr=0.0002, max_dataset_size=10, model='pix2pixHD', nThreads=2, n_blocks_global=9, n_blocks_local=3, n_clusters=10, n_downsample_E=4, n_downsample_global=4, n_layers_D=3, n_local_enhancers=1, name='maskGirl', ndf=64, nef=16, netG='global', ngf=64, niter=40, niter_decay=40, niter_fix_global=0, no_flip=True, no_ganFeat_loss=False, no_html=False, no_instance=True, no_lsgan=False, no_vgg_loss=False, norm='instance', num_D=2, output_nc=3, phase='train', pool_size=0, print_freq=1, resize_or_crop='scale_width', save_epoch_freq=10, save_latest_freq=640, serial_batches=False, temp_data=False, tf_log=False, use

In [7]:
data_loader = CreateDataLoader(opt)
dataset = data_loader.load_data()
dataset_size = len(data_loader)
print('#training images = %d' % dataset_size)
    
# start_epoch, epoch_iter = 1, 0
total_steps = (start_epoch-1) * dataset_size + epoch_iter
display_delta = total_steps % opt.display_freq
print_delta = total_steps % opt.print_freq
save_delta = total_steps % opt.save_latest_freq

CustomDatasetDataLoader
dataset [AlignedDataset] was created
#training images = 10


In [8]:
model = create_model(opt)
visualizer = Visualizer(opt)

GlobalGenerator(
  (model): Sequential(
    (0): ReflectionPad2d((3, 3, 3, 3))
    (1): Conv2d(3, 64, kernel_size=(7, 7), stride=(1, 1))
    (2): InstanceNorm2d(64, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
    (3): ReLU(inplace)
    (4): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (5): InstanceNorm2d(128, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
    (6): ReLU(inplace)
    (7): Conv2d(128, 256, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (8): InstanceNorm2d(256, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
    (9): ReLU(inplace)
    (10): Conv2d(256, 512, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (11): InstanceNorm2d(512, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
    (12): ReLU(inplace)
    (13): Conv2d(512, 1024, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (14): InstanceNorm2d(1024, eps=1e-05, momentum=0.1, affine=False, track_runn

create web directory ../checkpoints/maskGirl/web...


In [9]:
step = 0
for epoch in range(start_epoch, opt.niter + opt.niter_decay + 1):
    epoch_start_time = time.time()
    if epoch != start_epoch:
        epoch_iter = epoch_iter % dataset_size
    for i, data in enumerate(dataset, start=epoch_iter):
        iter_start_time = time.time()
        total_steps += opt.batchSize
        epoch_iter += opt.batchSize

        # whether to collect output images
        save_fake = total_steps % opt.display_freq == display_delta
        
        ############## Forward Pass ######################
        losses, generated = model(Variable(data['label']), Variable(data['inst']), 
            Variable(data['image']), Variable(data['feat']), infer=save_fake)
        
        # sum per device losses
        losses = [ torch.mean(x) if not isinstance(x, int) else x for x in losses ]
        loss_dict = dict(zip(model.module.loss_names, losses))

        # calculate final loss scalar
        loss_D = (loss_dict['D_fake'] + loss_dict['D_real']) * 0.5
        loss_G = loss_dict['G_GAN'] + loss_dict.get('G_GAN_Feat',0) + loss_dict.get('G_VGG',0)
        
        ############### Backward Pass ####################
        # update generator weights
        model.module.optimizer_G.zero_grad()
        loss_G.backward()
        model.module.optimizer_G.step()
        model.module.optimizer_D.zero_grad()
#         step += 1

        # update discriminator weights
        model.module.optimizer_D.zero_grad()
        loss_D.backward()
        model.module.optimizer_D.step()
#         step = 0
        #call(["nvidia-smi", "--format=csv", "--query-gpu=memory.used,memory.free"]) 

        ############## Display results and errors ##########
        ### print out errors
        if total_steps % opt.print_freq == print_delta:
            errors = {k: v.data[0] if not isinstance(v, int) else v for k, v in loss_dict.items()}
            t = (time.time() - iter_start_time) / opt.batchSize
            visualizer.print_current_errors(epoch, epoch_iter, errors, t)
            visualizer.plot_current_errors(errors, total_steps)

        ### display output images
        if save_fake:
            visuals = OrderedDict([('input_label', util.tensor2label(data['label'][0], opt.label_nc)),
                                   ('synthesized_image', util.tensor2im(generated.data[0])),
                                   ('real_image', util.tensor2im(data['image'][0]))])
            visualizer.display_current_results(visuals, epoch, total_steps)

        ### save latest model
        if total_steps % opt.save_latest_freq == save_delta:
            print('saving the latest model (epoch %d, total_steps %d)' % (epoch, total_steps))
            model.module.save('latest')            
            np.savetxt(iter_path, (epoch, epoch_iter), delimiter=',', fmt='%d')

        if epoch_iter >= dataset_size:
            break
       
    # end of epoch 
    iter_end_time = time.time()
    print('End of epoch %d / %d \t Time Taken: %d sec' %
          (epoch, opt.niter + opt.niter_decay, time.time() - epoch_start_time))

    ### save model for this epoch
    if epoch % opt.save_epoch_freq == 0:
        print('saving the model at the end of epoch %d, iters %d' % (epoch, total_steps))        
        model.module.save('latest')
        model.module.save(epoch)
        np.savetxt(iter_path, (epoch+1, 0), delimiter=',', fmt='%d')

    ### instead of only training the local enhancer, train the entire network after certain iterations
    if (opt.niter_fix_global != 0) and (epoch == opt.niter_fix_global):
        model.module.update_fixed_params()

    ### linearly decay learning rate after certain iterations
    if epoch > opt.niter:
        model.module.update_learning_rate()
        
torch.cuda.empty_cache()

/home/yaosy/anaconda3/envs/pytorch/lib/python3.6/site-packages/ipykernel_launcher.py:44: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number


(epoch: 1, iters: 1, time: 5.357) G_GAN: 6.186 G_GAN_Feat: 14.472 G_VGG: 9.809 D_real: 6.433 D_fake: 2.813 
(epoch: 1, iters: 2, time: 0.463) G_GAN: 12.395 G_GAN_Feat: 14.013 G_VGG: 9.051 D_real: 14.372 D_fake: 10.753 
(epoch: 1, iters: 3, time: 0.457) G_GAN: 13.088 G_GAN_Feat: 14.704 G_VGG: 9.136 D_real: 7.058 D_fake: 12.049 
(epoch: 1, iters: 4, time: 0.457) G_GAN: 10.806 G_GAN_Feat: 14.253 G_VGG: 9.082 D_real: 6.318 D_fake: 10.376 
(epoch: 1, iters: 5, time: 0.439) G_GAN: 4.000 G_GAN_Feat: 14.204 G_VGG: 8.435 D_real: 3.240 D_fake: 3.937 
(epoch: 1, iters: 6, time: 0.458) G_GAN: 4.714 G_GAN_Feat: 13.433 G_VGG: 8.668 D_real: 2.008 D_fake: 4.486 
(epoch: 1, iters: 7, time: 0.461) G_GAN: 3.238 G_GAN_Feat: 12.739 G_VGG: 8.177 D_real: 1.384 D_fake: 2.911 
(epoch: 1, iters: 8, time: 0.458) G_GAN: 2.015 G_GAN_Feat: 12.095 G_VGG: 8.261 D_real: 1.469 D_fake: 1.732 
(epoch: 1, iters: 9, time: 0.444) G_GAN: 1.503 G_GAN_Feat: 11.342 G_VGG: 7.916 D_real: 0.958 D_fake: 1.080 
(epoch: 1, iters: 10,

Exception ignored in: <bound method _DataLoaderIter.__del__ of <torch.utils.data.dataloader._DataLoaderIter object at 0x7fbe0a831630>>
Traceback (most recent call last):
  File "/home/yaosy/anaconda3/envs/pytorch/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 399, in __del__
    self._shutdown_workers()
  File "/home/yaosy/anaconda3/envs/pytorch/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 378, in _shutdown_workers
    self.worker_result_queue.get()
  File "/home/yaosy/anaconda3/envs/pytorch/lib/python3.6/multiprocessing/queues.py", line 337, in get
    return _ForkingPickler.loads(res)
  File "/home/yaosy/anaconda3/envs/pytorch/lib/python3.6/site-packages/torch/multiprocessing/reductions.py", line 151, in rebuild_storage_fd
    fd = df.detach()
  File "/home/yaosy/anaconda3/envs/pytorch/lib/python3.6/multiprocessing/resource_sharer.py", line 57, in detach


End of epoch 4 / 80 	 Time Taken: 5 sec


    with _resource_sharer.get_connection(self._id) as conn:
  File "/home/yaosy/anaconda3/envs/pytorch/lib/python3.6/multiprocessing/resource_sharer.py", line 87, in get_connection
    c = Client(address, authkey=process.current_process().authkey)
  File "/home/yaosy/anaconda3/envs/pytorch/lib/python3.6/multiprocessing/connection.py", line 493, in Client
    answer_challenge(c, authkey)
  File "/home/yaosy/anaconda3/envs/pytorch/lib/python3.6/multiprocessing/connection.py", line 737, in answer_challenge
    response = connection.recv_bytes(256)        # reject large message
  File "/home/yaosy/anaconda3/envs/pytorch/lib/python3.6/multiprocessing/connection.py", line 216, in recv_bytes
    buf = self._recv_bytes(maxlength)
  File "/home/yaosy/anaconda3/envs/pytorch/lib/python3.6/multiprocessing/connection.py", line 407, in _recv_bytes
    buf = self._recv(4)
  File "/home/yaosy/anaconda3/envs/pytorch/lib/python3.6/multiprocessing/connection.py", line 379, in _recv
    chunk = read(handl

...

(epoch: 65, iters: 1, time: 0.451) G_GAN: 0.662 G_GAN_Feat: 6.309 G_VGG: 4.128 D_real: 0.410 D_fake: 0.437 
(epoch: 65, iters: 2, time: 0.461) G_GAN: 1.070 G_GAN_Feat: 5.449 G_VGG: 5.016 D_real: 0.449 D_fake: 0.237 
(epoch: 65, iters: 3, time: 0.444) G_GAN: 0.690 G_GAN_Feat: 4.581 G_VGG: 4.655 D_real: 0.355 D_fake: 0.413 
(epoch: 65, iters: 4, time: 0.449) G_GAN: 0.778 G_GAN_Feat: 4.125 G_VGG: 4.456 D_real: 0.337 D_fake: 0.344 
(epoch: 65, iters: 5, time: 0.432) G_GAN: 0.783 G_GAN_Feat: 4.860 G_VGG: 4.688 D_real: 0.428 D_fake: 0.356 
(epoch: 65, iters: 6, time: 0.431) G_GAN: 0.806 G_GAN_Feat: 5.709 G_VGG: 5.177 D_real: 0.289 D_fake: 0.376 
(epoch: 65, iters: 7, time: 0.431) G_GAN: 0.820 G_GAN_Feat: 5.266 G_VGG: 4.881 D_real: 0.351 D_fake: 0.336 
(epoch: 65, iters: 8, time: 0.429) G_GAN: 0.694 G_GAN_Feat: 5.148 G_VGG: 5.010 D_real: 0.323 D_fake: 0.424 
(epoch: 65, iters: 9, time: 0.424) G_GAN: 1.026 G_GAN_Feat: 5.727 G_VGG: 5.171 D_real: 0.344 D_fake: 0.240 
(epoch: 65, iters: 10, time:

(epoch: 72, iters: 4, time: 0.449) G_GAN: 0.884 G_GAN_Feat: 5.119 G_VGG: 4.716 D_real: 0.317 D_fake: 0.286 
(epoch: 72, iters: 5, time: 0.451) G_GAN: 0.937 G_GAN_Feat: 5.071 G_VGG: 4.647 D_real: 0.314 D_fake: 0.260 
(epoch: 72, iters: 6, time: 0.439) G_GAN: 0.872 G_GAN_Feat: 5.493 G_VGG: 4.797 D_real: 0.303 D_fake: 0.297 
(epoch: 72, iters: 7, time: 0.451) G_GAN: 0.725 G_GAN_Feat: 5.250 G_VGG: 4.630 D_real: 0.459 D_fake: 0.397 
(epoch: 72, iters: 8, time: 0.449) G_GAN: 0.769 G_GAN_Feat: 4.503 G_VGG: 4.529 D_real: 0.334 D_fake: 0.359 
(epoch: 72, iters: 9, time: 0.427) G_GAN: 0.695 G_GAN_Feat: 3.926 G_VGG: 4.328 D_real: 0.352 D_fake: 0.400 
(epoch: 72, iters: 10, time: 0.443) G_GAN: 0.817 G_GAN_Feat: 4.774 G_VGG: 4.550 D_real: 0.441 D_fake: 0.325 
End of epoch 72 / 80 	 Time Taken: 5 sec
(epoch: 73, iters: 1, time: 0.435) G_GAN: 0.618 G_GAN_Feat: 4.037 G_VGG: 4.437 D_real: 0.325 D_fake: 0.469 
(epoch: 73, iters: 2, time: 0.436) G_GAN: 0.850 G_GAN_Feat: 5.485 G_VGG: 4.684 D_real: 0.300 D

(epoch: 79, iters: 8, time: 0.437) G_GAN: 0.813 G_GAN_Feat: 5.050 G_VGG: 4.583 D_real: 0.346 D_fake: 0.327 
(epoch: 79, iters: 9, time: 0.422) G_GAN: 0.802 G_GAN_Feat: 5.056 G_VGG: 4.443 D_real: 0.334 D_fake: 0.329 
(epoch: 79, iters: 10, time: 0.428) G_GAN: 0.828 G_GAN_Feat: 4.623 G_VGG: 4.378 D_real: 0.463 D_fake: 0.318 
End of epoch 79 / 80 	 Time Taken: 5 sec


Exception ignored in: <bound method _DataLoaderIter.__del__ of <torch.utils.data.dataloader._DataLoaderIter object at 0x7fbe092b8a58>>
Traceback (most recent call last):
  File "/home/yaosy/anaconda3/envs/pytorch/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 399, in __del__
    self._shutdown_workers()
  File "/home/yaosy/anaconda3/envs/pytorch/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 378, in _shutdown_workers
    self.worker_result_queue.get()
  File "/home/yaosy/anaconda3/envs/pytorch/lib/python3.6/multiprocessing/queues.py", line 337, in get
    return _ForkingPickler.loads(res)
  File "/home/yaosy/anaconda3/envs/pytorch/lib/python3.6/site-packages/torch/multiprocessing/reductions.py", line 151, in rebuild_storage_fd
    fd = df.detach()
  File "/home/yaosy/anaconda3/envs/pytorch/lib/python3.6/multiprocessing/resource_sharer.py", line 58, in detach
    return reduction.recv_handle(conn)
  File "/home/yaosy/anaconda3/envs/pytorch/lib/pytho

(epoch: 80, iters: 1, time: 0.429) G_GAN: 0.787 G_GAN_Feat: 4.355 G_VGG: 4.175 D_real: 0.431 D_fake: 0.336 
(epoch: 80, iters: 2, time: 0.425) G_GAN: 0.735 G_GAN_Feat: 4.456 G_VGG: 4.393 D_real: 0.371 D_fake: 0.374 
(epoch: 80, iters: 3, time: 0.419) G_GAN: 0.857 G_GAN_Feat: 5.514 G_VGG: 4.566 D_real: 0.330 D_fake: 0.295 
(epoch: 80, iters: 4, time: 0.433) G_GAN: 0.686 G_GAN_Feat: 4.460 G_VGG: 4.313 D_real: 0.433 D_fake: 0.415 
(epoch: 80, iters: 5, time: 0.433) G_GAN: 0.737 G_GAN_Feat: 3.740 G_VGG: 4.103 D_real: 0.387 D_fake: 0.377 
(epoch: 80, iters: 6, time: 0.447) G_GAN: 0.784 G_GAN_Feat: 5.357 G_VGG: 4.982 D_real: 0.331 D_fake: 0.373 
(epoch: 80, iters: 7, time: 0.430) G_GAN: 0.731 G_GAN_Feat: 4.257 G_VGG: 4.109 D_real: 0.375 D_fake: 0.368 
(epoch: 80, iters: 8, time: 0.421) G_GAN: 0.732 G_GAN_Feat: 4.672 G_VGG: 4.168 D_real: 0.321 D_fake: 0.371 
(epoch: 80, iters: 9, time: 0.442) G_GAN: 0.887 G_GAN_Feat: 5.901 G_VGG: 4.985 D_real: 0.254 D_fake: 0.302 
(epoch: 80, iters: 10, time: